<a href="https://colab.research.google.com/github/edwarddavis1/CharMem/blob/main/src/basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Clone the repo (requires PAT as it's currently a private repo)

from getpass import getpass

token = getpass("Enter GitHub PAT")

!git clone https://{token}@github.com/edwarddavis1/CharMem.git

Enter GitHub PAT··········
Cloning into 'CharMem'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 30 (delta 8), reused 27 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 789.17 KiB | 27.21 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [9]:
%cd CharMem
!pip install .

/content/CharMem
Processing /content/CharMem
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached chromadb-1.0.11-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_chroma-0.2.4-py3-none-any.whl.metadata (1.1 kB)
  Using cached langchain_community-0.3.24-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_huggingface-0.2.0-py3-none-any.whl.metadata (941 bytes)
  Using cached langchain_openai-0.3.18-py3-none-any.whl.metadata (2.3 kB)
  Using cached llama_cpp_python-0.3.9.tar.gz (67.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pypdf-5.5.0-py3-none-any.whl.metadata (7.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [7]:
# Enter the working directory
%cd CharMem/src

/content/CharMem/src


In [10]:
%pwd

'/content/CharMem/src'

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import os
import shutil


In [12]:
# LOAD THE DATA

DATA_PATH = "../data/books"
book = "Harry-Potter-and-the-Philosophers-Stone"
loader = PyPDFLoader(DATA_PATH + "/" + book + ".pdf")
pages = loader.load()

# # Each "page" is a LangChain Document with .page_content
# for i, page in enumerate(pages[:3]):
#     print(f"\n--- Page {i + 1} ---\n")
#     print(page.page_content[:500])


In [13]:
# CHUNK THE TEXT

chunk_size = 1000
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True,
)
chunks = splitter.split_documents(pages)

page = chunks[50]
print(page.page_content)

— CHAPTER TWO — 
 
The V anishing Glass 
 
Nearly ten years had passed since the Dursleys had woken up to 
find their nephew on the front step, but Privet Drive had hardly 
changed at all. The sun rose on the same tidy front gardens and lit 
up the brass number four on the Dursleys’ front door; it crept into 
their living-room, which was almost exactly the same as it had 
been on the night when Mr Dursley had seen that fateful news 
report about the owls. Only the photographs on the mantelpiece 
really showed how much time had passed. Ten years ago, there 
had been lots of pictures of what looked like a large pink beach 
ball wearing different-coloured bobble hats – but Dudley Dursley 
was no longer a baby , and now the photographs showed a large, 
blond boy riding his first bicycle, on a roundabout at the fair, 
playing a computer game with his father, being hugged and kissed 
by his mother. The room held no sign at all that another boy lived 
in the house, too.


In [15]:
# USE VECTOR DATABASE TO EMBED EACH OF THE CHUNKS

# Free model from hugging face
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create vector database by embedding each of the chunks using the
#  specified embedding model
CHROMA_PATH = "chroma"

# Remove previous database if making a new one
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

# Create vector database
db = Chroma.from_documents(chunks, embedding_function, persist_directory=CHROMA_PATH)



In [16]:
# RAG

# Load the database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the database
query_text = "Who is Hermione Granger?"
results = db.similarity_search_with_relevance_scores(query_text, k=3)

# # Filter
# if len(results) == 0 or results[0][1] < 0.5:
#     print("Unable to find good results")

# View the retrieval
retrieval = "\n\n---\n\n".join([page.page_content for page, _ in results])
print(retrieval)


‘Five, you mean, once she’s taken off Hermione’s.’ 
‘Good of her to get us out of trouble like that,’ Ron admitted. 
‘Mind you, we did save her.’ 
‘She might not have needed saving if we hadn’t locked the thing 
in with her,’ Harry reminded him. 
They had reached the portrait of the Fat Lady . 
‘Pig snout,’ they said and entered. 
The common room was packed and noisy . Everyone was eating 
the food that had been sent up . Hermione, however, stood alone 
by the door, waiting for them. There was a very embarrassed 
pause. Then, none of them looking at each other, they all said 
‘Thanks’, and hurried off to get plates. 
But from that moment on, Hermione Granger became their 
friend. There are some things you can’t share without ending up 
liking each other, and knocking ou t a twelve-foot mountain troll 
is one of them.

---

Hermione Granger, by the way . Students up at the school. An’ this 
is Ronan, you two. He’s a centaur.’ 
‘We’d noticed,’ said Hermione faintly . 
‘Good evening,’ sai

In [17]:
# CREATE RESPONSE
PROMPT_TEMPLATE = """
You are a helpful book assistant. Given the following excerpts from a novel, answer the user’s question as clearly and concisely as possible, using only the provided text.

Context:
{context}

Question: {query}

Answer:"""


prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=retrieval, query=query_text)


In [18]:
# LOAD LLM

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "tiiuae/falcon-rw-1b"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
# GENERATE RESPONSE

def ask_local_llm(prompt, max_new_tokens=150):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Run the local model
response = ask_local_llm(prompt)
print(response)
